<a href="https://colab.research.google.com/github/StevenVuong/MSc_Project/blob/master/v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies
!pip install deepbrain; # semi-colon to hide the output
!pip install pydicom;

In [2]:
from google.colab import drive

# mount google drive into google colab
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
    
# go to where the data is
print (os.listdir())
os.chdir('gdrive/My Drive/msc_project')
os.listdir()

['.config', 'gdrive', 'sample_data']


['t1_scan',
 'T1_SAG_SIEMEN_3T_CLEAN_5_29_2019.csv',
 'T1_SAG_SIEMEN_3T_CLEAN_1',
 'T1_SAG_SIEMEN_3T_CLEAN_5_29_2019.gsheet']

In [4]:
import pandas as pd
# https://www.kaggle.com/sentdex/first-pass-through-data-w-3d-convnet
patients = os.listdir('T1_SAG_SIEMEN_3T_CLEAN_1') # get all patients ID's in scan
patient_df = pd.read_csv('T1_SAG_SIEMEN_3T_CLEAN_5_29_2019.csv') # get dataframe too to cross reference

patient_df.head() # so we have a dataframe of our patients' data

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,1130198,75422,GenCohort Unaff,M,73,1,MRI,MPRAGE GRAPPA,Original,11/13/2018,DCM,5/07/2019
1,1130190,75414,GenCohort Unaff,F,73,1,MRI,Sag MPRAGE GRAPPA,Original,12/13/2018,DCM,4/24/2019
2,1130191,75414,GenCohort Unaff,F,73,1,MRI,Sag MPRAGE GRAPPA,Original,12/13/2018,DCM,4/24/2019
3,1125041,74375,GenCohort Unaff,F,59,1,MRI,MPRAGE_GRAPPA,Original,9/06/2018,DCM,4/24/2019
4,1003469,72138,GenCohort Unaff,F,55,1,MRI,MPRAGE GRAPPA,Original,2/19/2018,DCM,4/24/2019


In [0]:
# Map GenCohort to regular PD and Controls
patient_df['Group'] = patient_df['Group'].replace({'GenCohort PD':'PD', 'GenCohort Unaff':'Control'})

In [0]:
def get_grappa_dir(path):
  # get the file ending with 'GRAPPA', would need to accomodate this for grappa also
  for next_path in os.listdir(path):
    if (next_path.split("_")[-1] == 'GRAPPA'): # for the t1 weighted
      return next_path

In [0]:
def get_dcm_s(path):
  # get the path beginning with S, so doesn't clash with GZ File
  for next_path in os.listdir(path):
    if (next_path[0] == 'S'):
      return next_path

In [0]:
def get_img_no(path):
  # get the image identification numberm any image will do for this so take first
  image_number = None
  for image_file in os.listdir(path):
    image_number = int(image_file.split("_")[-1][1:-4]) # index to get the ID
   
  return image_number

In [0]:
def filename_sort(filename):
    
    # split by underlines and delimiter
    split_line = filename.split("_")
    int_return = int(split_line[-3])
    
    return int_return

In [10]:
import pydicom
import numpy as np

cwd = os.getcwd()
print ("Current Working Dir: %s " % cwd)

total_slices = [] # build all the slices

for patient in patients[:5]:
  # label = patient_df.get_value(patient, 'Subject') # cannot go by patient, must get the ID
  path = cwd + '/' + 'T1_SAG_SIEMEN_3T_CLEAN_1/' + patient # get to the GRAPPA 
  path = path + '/' + get_grappa_dir(path)
  path = path + '/' + os.listdir(path)[-1] # get the most recent scan for patient
  path = path + '/' + get_dcm_s(path)
  
  # get information related around the image
  image_number = get_img_no(path)
  image_row = patient_df.loc[patient_df['Image Data ID'] == image_number] # relate to df
  image_sex = image_row.Sex.values[0]
  image_group = image_row.Group.values[0]
  image_age = image_row.Age.values[0]
  
  # create image object and append to total info
  image_info = [image_number, image_sex, image_group, image_age]
  
  print ("Sex: %s, Age: %s, Group: %s " % (image_sex, image_age, image_group))
  
  # get files and sort them in order
  dcm_files = os.listdir(path)
  dcm_files = sorted(dcm_files, key=lambda filename: filename_sort(filename)) # some have length 3
  
  slices = []
  # loop through slices and build the array
  for dcm_file in dcm_files:
    path_to_file = path + '/' + dcm_file
    slices.append(pydicom.read_file(path_to_file).pixel_array)
  slices = np.array(slices)[15:175, :, :]
  total_slices.append([slices, image_info])
  
  print (np.shape(slices)) # each patient has different number of slices, trim it to [15:175, 30:230, 30:230]

Current Working Dir: /content/gdrive/My Drive/msc_project 
Sex: F, Age: 50, Group: PD 
(160, 256, 240)
Sex: M, Age: 53, Group: PD 
(160, 256, 240)
Sex: M, Age: 56, Group: PD 
(160, 256, 240)
Sex: M, Age: 44, Group: PD 
(160, 256, 240)
Sex: M, Age: 64, Group: PD 
(160, 256, 240)


In [11]:
from deepbrain import Extractor
import nibabel as nb

total_slices_processed = []
slice_info = []

for total_slice in total_slices:
  # deal with mixed slice information
  slices = total_slice[0]
  slice_info.append(total_slice[1])
  
  # transform into axial view
  slice_axial = slices.transpose((1,2,0))
  
  # initialise skull stripper
  ext = Extractor()

  # get probability of part of image being brain tissue or not
  prob = ext.run(slice_axial)
  mask = prob > 1e-3 # mask can be obtained as:
  slice_axial[~mask] = 0 # apply mask
  
  slice_axial = slice_axial[30:230, 30:230, :] # trim blank ones
  total_slices_processed.append(slice_axial) # add original
  
  # flip images and add to total processed arrays
  flipped_slices = [np.flip(sl,1) for sl in slice_axial]
  total_slices_processed.append(flipped_slices)
  slice_info.append(total_slice[1]) # add info twice
    
  print ("Regular Shape: %s " % (np.shape(slice_axial), ))
  print ("Flipped Shape: %s " % (np.shape(flipped_slices), ))

Instructions for updating:
Use tf.gfile.GFile.
Regular Shape: (200, 200, 160) 
Flipped Shape: (200, 200, 160) 
Regular Shape: (200, 200, 160) 
Flipped Shape: (200, 200, 160) 
Regular Shape: (200, 200, 160) 
Flipped Shape: (200, 200, 160) 
Regular Shape: (200, 200, 160) 
Flipped Shape: (200, 200, 160) 
Regular Shape: (200, 200, 160) 
Flipped Shape: (200, 200, 160) 


In [12]:
total_slices_processed = np.array(total_slices_processed) # turn into array
total_slices_processed = np.expand_dims(total_slices_processed, axis=1) # expand dimensions

np.shape(total_slices_processed)

(10, 200, 200, 160)

In [0]:
# need to build the y-data set (PD or Healthy)
# then split into test and training set
# then try run through a basic model

In [14]:
from keras.utils import to_categorical

# build y-outputs
diagnosis = [s[2] for s in slice_info] # we got our y-values
diagnosis = [1 if s=='PD' else s for s in diagnosis]
diagnosis = [0 if s=='Control' else s for s in diagnosis]

y_output = to_categorical(diagnosis, 2) # convert to something categorical with keras util
y_output = np.array(y_output)

print (y_output)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


Using TensorFlow backend.


In [15]:
from sklearn.model_selection import train_test_split

# split into training and test set
X_train, X_test, y_train, y_test = train_test_split(total_slices_processed, y_output, test_size=0.2, shuffle=True)

np.shape(X_train)

(8, 200, 200, 160)

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils, generic_utils
from keras.layers import LeakyReLU

In [17]:
model = Sequential()

model.add(Convolution3D(filters=32, kernel_size=4, strides=1, 
                        padding='same', input_shape=(1, 200,200,160), activation=None)) # or should activation be linear?
model.add(LeakyReLU(alpha=0.01)) # set to 0.01
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=1, padding='same'))

model.add(Convolution3D(filters=64, kernel_size=4, strides=1, 
                        padding='same', activation=None))
model.add(LeakyReLU(alpha=0.01)) 
model.add(MaxPooling3D(pool_size=(4, 4, 4), strides=1, padding='same'))

model.add(Convolution3D(filters=128, kernel_size=4, strides=1, 
                        padding='same', activation=None))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling3D(pool_size=(4, 4, 4), strides=1, padding='same'))
model.add(Flatten())
model.add(Dense(64, activation=None))
model.add(LeakyReLU(alpha=0.01))

model.add(Dense(16, activation=None))
model.add(LeakyReLU(alpha=0.01))

model.add(Dense(2, activation='softmax'))

model.compile(optimizer=Adam(lr=0.00005), loss='categorical_crossentropy',metrics = ['accuracy']) # metrics=['categorical_accuracy']

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 1, 200, 200, 32)   327712    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1, 200, 200, 32)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 1, 200, 200, 32)   0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 1, 200, 200, 64)   131136    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1, 200, 200, 64)   0         
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 1, 200, 200, 64)   0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 200, 200, 128)  524416    
__________

In [19]:
model.fit(x=X_train, y=y_train, batch_size=None, epochs=100, verbose=2,
          validation_data=(X_test, y_test), shuffle=True)

score = model.evaluate(X_test, Y_test, batch_size=None, show_accuracy=True)
print('Test score:', score[0])
print('Test accuracy:', score[1])

ValueError: ignored

In [0]:
# https://github.com/MinhazPalasara/keras/blob/master/examples/shapes_3d_cnn.py
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(X_train, Y_train, batch_size=10 , nb_epoch=nb_epoch, show_accuracy=True, verbose=2,
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, batch_size=batch_size, show_accuracy=True)

In [0]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets # interactive plots
import matplotlib.pyplot as plt
%matplotlib inline

def g(i): # basic slideshow plot to get an idea of the effectiveness of the mask itself
    plt.figure(figsize=(15,8)) # make plot larger
    plt.imshow(total_slices_processed[1][i])
    plt.show()
    return None
  
interact(g, i=widgets.IntSlider(min=0,max=(len(total_slices_processed[1])-1),step=1,value=65)); # plots our axial view, this is it